In [1]:
%pip install --upgrade networkx[default]
%pip install pyinform

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
# reqruied libraries

import glob
import re
import os
import time
import datetime

import pandas as pd
import numpy as np
import networkx as nx
import pyinform

# local functions import

import transfer_entropy_functions
import url_identifier

In [3]:
# s3 specific libraries
import s3fs
import boto3
s3 = s3fs.S3FileSystem(anon=False)

In [4]:
# Running with S3 (inside AWS) ?
RUNNING_IN_S3 = True

In [18]:
# Directories
SAVE_OUTPUT_FILES = True
OUTPUT_DIR = './OUTPUT'

UPLOAD_FILES = False
UPLOAD_S3_BUCKET_NAME = 'mips-main'
UPLOAD_S3_BUCKET_SUBFOLDER = 'initial_data_collection/processed_data/actors_and_messages/v4'

#PC_INPUT_BRANDWATCH_DIR = 'C:/STUFF/RESEARCH/Brandwatch/TEST'
PC_INPUT_BRANDWATCH_DIR = 'C:/STUFF/RESEARCH/Brandwatch/DATA/MainQuery/All'
S3_INPUT_BRANDWATCH_DIR = 's3://mips-main/initial_data_collection/raw_data/brandwatch'
INPUT_BRANDWATCH_DIR = S3_INPUT_BRANDWATCH_DIR if RUNNING_IN_S3 else PC_INPUT_BRANDWATCH_DIR

INPUT_4CHAN_DIR = 's3://mips-main/initial_data_collection/TE_ready_data/V2'

INPUT_REDDIT_DIR = 's3://mips-main/initial_data_collection/TE_ready_data/V2'

S3_NEWS_DOMAINS_CSV_FILE = 's3://mips-main/initial_data_collection/news_outlets_v3.csv'
PC_NEWS_DOMAINS_CSV_FILE = 'C:/STUFF/RESEARCH/Brandwatch/DATA/news_outlets_v3.csv'
NEWS_DOMAINS_CSV_FILE = S3_NEWS_DOMAINS_CSV_FILE if RUNNING_IN_S3 else PC_NEWS_DOMAINS_CSV_FILE

S3_NEWS_DOMAIN_TUFM_FILE = "s3://mips-main-2/UFTM_classification-v2/news_table-v2-UT60-FM5.csv"
#S3_NEWS_DOMAIN_TUFM_FILE = 's3://mips-main/initial_data_collection/processed_data/news_sources/news_table-v2-UT60-FM5.csv'
PC_NEWS_DOMAIN_TUFM_FILE = 'C:/STUFF/RESEARCH/Brandwatch/DATA/news_table-v2-UT60-FM5.csv' 
NEWS_DOMAIN_TUFM_FILE = S3_NEWS_DOMAIN_TUFM_FILE if RUNNING_IN_S3 else PC_NEWS_DOMAIN_TUFM_FILE

In [10]:
OUTPUT_PLATFORM_COUNTS_FILE     = OUTPUT_DIR + "/platform_counts_info.csv"
OUTPUT_USERS_FILE               = OUTPUT_DIR + "/users.csv"
OUTPUT_RETWEET_NETWORK_FILE     = OUTPUT_DIR + "/retweet_network.csv"
OUTPUT_ALL_USER_COMM_SIZES_FILE = OUTPUT_DIR + "/all_user_comm_sizes.csv"
OUTPUT_PLATFORM_ACTORS_FILE     = OUTPUT_DIR + "/plat_actors.csv"
OUTPUT_INDV_ACTORS_FILE         = OUTPUT_DIR + "/indv_actors.csv"
OUTPUT_COMM_INFO_FILE           = OUTPUT_DIR + "/comm_info.csv"
OUTPUT_COMM_ACTORS_FILE         = OUTPUT_DIR + "/comm_actors.csv"
OUTPUT_ACTORS_FILE              = OUTPUT_DIR + "/actors.csv"
OUTPUT_ALL_OSN_MSGS_FILE        = OUTPUT_DIR + "/all_osn_msgs.csv"

In [11]:
# Parameters
SAMPLE_FREQUENCY = '6H'

WINDOW_SHIFT_BY_DAYS = 1
INIT_WINDOW_SIZE = datetime.timedelta(days=4)
IS_GROWING_WINDOW = True # True if Growing, False if Moving

START_DATE = datetime.datetime(2018,3,1)
END_DATE = datetime.datetime(2018,5,1)

#skip_strings = {'add', 'al', 'au', 'ca', 'com', 'es', 'in', 'is', 'it', 'ms', 'my', 'net', 'news', 'org', 'rs', 'st', 'tv', 'uk', 'us', 'co'}

MIN_COMM_SIZE = 500
MIN_PLAT_SIZE = 50

#KNOWN_PLATFORMS = {'twitter.com', 'tumblr.com', 'youtube.com', 'reddit.com', '4chan.org', 'facebook.com', 'gab.com'}

REQUIRED_COLUMNS = ['datetime','source_msg_id', 'source_user_id', 'content', 'title', 'parent_source_msg_id', 'parent_source_user_id', 'platform', 'article_url']


In [12]:
# All columns:
reddit_submissions_column_dict = {'archived' : 'archived', 'author' : 'source_user_id', 'call_to_action' : 'call_to_action', 'can_gild' : 'can_gild', 'contest_mode' : 'contest_mode',
       'created_utc' : 'created_utc', 'datetime' : 'datetime', 'domain' : 'domain', 'gilded' : 'gilded', 'hidden' : 'hidden', 'hide_score' : 'hide_score',
       'id' : 'source_msg_id', 'is_crosspostable' : 'is_crosspostable', 'is_reddit_media_domain' : 'is_reddit_media_domain', 'is_self' : 'is_self',
       'is_video' : 'is_video', 'locked' : 'locked', 'no_follow' : 'no_follow', 'num_comments' : 'num_comments', 'num_crossposts' : 'num_crossposts',
       'over_18' : 'over_18', 'permalink' : 'permalink', 'pinned' : 'pinned', 'score' : 'score', 'selftext' : 'content', 'send_replies' : 'send_replies',
       'spoiler' : 'spoiler', 'stickied' : 'stickied', 'subreddit' : 'subreddit', 'subreddit_id' : 'subreddit_id',
       'subreddit_subscribers' : 'subreddit_subscribers', 'subreddit_type' : 'subreddit_type', 'thumbnail' : 'thumbnail', 'title' : 'title', 'url' : 'url'}


reddit_comments_column_dict = {'author' : 'source_user_id', 'body' : 'content', 'can_gild' : 'can_gild', 'controversiality' : 'controversiality', 'created_utc' : 'created_utc',
       'datetime' : 'datetime', 'edited' : 'edited', 'id' : 'source_msg_id', 'is_submitter' : 'is_submitter', 'link_id' : 'link_id', 'no_follow' : 'no_follow', 
       'parent_id' : 'parent_source_msg_id', 'permalink' : 'permalink', 'retrieved_on' : 'retrieved_on', 'score' : 'score', 'send_replies' : 'send_replies',
       'stickied' : 'stickied', 'subreddit' : 'subreddit', 'subreddit_id' : 'subreddit_id', 'subreddit_type' : 'subreddit_type'}


fourchan_column_dict = {'archived' : 'archived', 'archived_on' : 'archived_on', 'bumplimit' : 'bumplimit', 'capcode' : 'capcode', 'closed' :'closed', 'com' : 'content',
       'country' : 'country', 'country_name' : 'country_name', 'ext' : 'ext', 'extracted_poster_id' : 'source_user_id', 'filedeleted' : 'filedeleted',
       'filename': 'filename', 'fsize' : 'fsize', 'h' : 'h', 'id' : 'id', 'imagelimit' : 'imagelimit', 'images' : 'images', 'm_img' : 'm_img', 'md5' : 'md5',
       'name' : 'name', 'no' : 'source_msg_id', 'now' : 'now', 'perspectives' : 'perspectives', 'replies' : 'replies', 'semantic_url' : 'semantic_url',
       'since4pass' : 'since4pass', 'spoiler' : 'spoiler', 'sticky' : 'sticky', 'sub' : 'sub', 'tail_size' : 'tail_size', 'tim' : 'tim', 'time' : 'time',
       'datetime' : 'datetime', 'datetime_dateformat' : 'datetime_dateformat', 'tn_h' : 'tn_h', 'tn_w' : 'tn_w', 'troll_country' : 'troll_country',
       'unique_ips' : 'unique_ips', 'w' : 'w', 'xa18' : 'xa18', 'xa19l' : 'xa19l', 'xa19s' : 'xa19s', 'xh17' : 'xh17', 'xh17c' : 'xh17c'}


brandwatch_column_dict={'Date':'datetime', 'Author': 'source_user_id', 'Full Text':'content', 'Title':'title', 
                        'Thread Id': 'parent_source_msg_id', 'Thread Author': 'parent_source_user_id', 
                        'Domain':'platform', 'Expanded URLs':'article_url', 'Url': 'source_msg_id'}

# Strip unwanted columns:
reddit_submissions_column_dict = (lambda columns_dict: {k:columns_dict[k] for k in columns_dict if columns_dict[k] in REQUIRED_COLUMNS})(reddit_submissions_column_dict)
reddit_comments_column_dict = (lambda columns_dict: {k:columns_dict[k] for k in columns_dict if columns_dict[k] in REQUIRED_COLUMNS})(reddit_comments_column_dict)
fourchan_column_dict = (lambda columns_dict: {k:columns_dict[k] for k in columns_dict if columns_dict[k] in REQUIRED_COLUMNS})(fourchan_column_dict)
brandwatch_column_dict = (lambda columns_dict: {k:columns_dict[k] for k in columns_dict if columns_dict[k] in REQUIRED_COLUMNS})(brandwatch_column_dict)

In [11]:
reddit_submissions_column_dict

{'author': 'source_user_id',
 'datetime': 'datetime',
 'id': 'source_msg_id',
 'selftext': 'content',
 'title': 'title'}

In [13]:
reddit_comments_column_dict

{'author': 'source_user_id',
 'body': 'content',
 'datetime': 'datetime',
 'id': 'source_msg_id',
 'parent_id': 'parent_source_msg_id'}

In [14]:
fourchan_column_dict

{'com': 'content',
 'extracted_poster_id': 'source_user_id',
 'no': 'source_msg_id',
 'datetime': 'datetime'}

In [15]:
brandwatch_column_dict

{'Date': 'datetime',
 'Author': 'source_user_id',
 'Full Text': 'content',
 'Title': 'title',
 'Thread Id': 'parent_source_msg_id',
 'Thread Author': 'parent_source_user_id',
 'Domain': 'platform',
 'Expanded URLs': 'article_url',
 'Url': 'source_msg_id'}

In [16]:
set(REQUIRED_COLUMNS).difference(list(fourchan_column_dict.values()))

{'article_url',
 'parent_source_msg_id',
 'parent_source_user_id',
 'platform',
 'title'}

In [17]:
def read_domain_to_class_classification():
    tufmdf = pd.read_csv(NEWS_DOMAIN_TUFM_FILE)
    tufmdf.rename(columns={'Domain':'news_domain'}, inplace=True)
    assert set(tufmdf['tufm_class'].unique()) == {'TF', 'TM', 'UF', 'UM'}, "News domains should belong to one of the classes in {'TF', 'TM', 'UF', 'UM'}"
    news_domain_to_tufm_class = tufmdf.set_index('news_domain')['tufm_class'].to_dict()
    del tufmdf
    return news_domain_to_tufm_class

news_domain_to_tufm_class = read_domain_to_class_classification()

In [20]:
news_domain_identifier = url_identifier.NewsDomainIdentifier(news_domain_to_tufm_class.keys())
news_domain_identifier

In [22]:
df = pd.read_csv("s3://" + s3.glob(os.path.join(INPUT_4CHAN_DIR, "fourchan*.csv*"))[0], usecols=list(fourchan_column_dict.keys()))
df.rename(columns=fourchan_column_dict, inplace=True)
df['platform'] = '4chan'
df['article_url'] = df['content'].apply(lambda x:  news_domain_identifier.find_all_matches(x) )
df['article_count'] = df['article_url'].apply(lambda x: len(x))
df['title'] = None
df['parent_source_msg_id'] = None
df['parent_source_user_id'] = None
df

,content,source_user_id,source_msg_id,datetime,platform,article_url,article_count,title,parent_source_msg_id,parent_source_user_id
0,ELECTRONIC messages sent to Moscow on the day ...,w0ABfHYp,167211216,2018-04-08 12:16:09.000,4chan,[bbc.com],1,None,None,None
1,The United Nations Security Council (UNSC) has...,9gTf6hzK,166870849,2018-04-05 19:29:42.000,4chan,[rt.com],1,None,None,None
2,The United Nations Security Council (UNSC) has...,9gTf6hzK,166870849,2018-04-05 19:29:42.000,4chan,[rt.com],1,None,None,None
3,The United Nations Security Council (UNSC) has...,9gTf6hzK,166870849,2018-04-05 19:29:42.000,4chan,[rt.com],1,None,None,None
4,The United Nations Security Council (UNSC) has...,9gTf6hzK,166870849,2018-04-05 19:29:42.000,4chan,[rt.com],1,None,None,None
...,...,...,...,...,...,...,...,...,...,...
773,ELECTRONIC messages sent to Moscow on the day ...,w0ABfHYp,167211216,2018-04-08 12:16:09.000,4chan,[bbc.com],1,None,None,None
774,ELECTRONIC messages sent to Moscow on the day ...,w0ABfHYp,167211216,2018-04-08 12:16:09.000,4chan,[bbc.com],1,None,None,None
775,ELECTRONIC messages sent to Moscow on the day ...,w0ABfHYp,167211216,2018-04-08 12:16:09.000,4chan,[bbc.com],1,None,None,None
776,ELECTRONIC messages sent to Moscow on the day ...,w0ABfHYp,167211216,2018-04-08 12:16:09.000,4chan,[bbc.com],1,None,None,None


In [ ]:
df[df['article_count'] < 1]

In [ ]:
news_domain_to_tufm_class.keys()

In [ ]:
# dfcomm.iloc[:100].apply(lambda row: row['parent_source_msg_id'], axis=1)

In [ ]:
set(brandwatch_column_dict.keys())

In [ ]:
set(['Date','Author','Full Text', 'Title','Thread Id', 'Thread Author','Domain', 'Expanded URLs','Url'])

In [ ]:
# dfposts[dfposts['source_msg_id'] == 't3_8hn80w']

#usecols=['Date','Author','Full Text', 'Title','Thread Id', 'Thread Author','Domain', 'Expanded URLs','Url']

In [ ]:
def read_4chan_data(data_directory, filter_platform_domain_set, is_using_s3):
    """
    Reads data from all the csv files in the given directory
    :param data_directory: Path to the directory that contains the csv files
    :type data_directory: str
    :return: pandas Dataframe that contains all the data from all csv files
    :rtype: pd.Dataframe
    """
    data_files = []
    if is_using_s3:
        data_files = s3.glob(os.path.join(data_directory, "*.csv*"))
    else:
        data_files = glob.glob(os.path.join(data_directory, "*.csv*"))
    prefix_path = ''
    if is_using_s3:
        prefix_path = 's3://'
    print(data_files)
    df_list = []
    for idx, file in enumerate(data_files):
        print(f"Reading {idx + 1} of {len(data_files)} files.\nFile name: {file}")
        df = pd.read_csv(prefix_path + data_files[idx], skiprows=6, parse_dates=['Date'],
                         dtype={'Twitter Author ID': str, 'Author':str,
                                'Full Text':str, 'Title':str,
                                'Thread Id':str, 'Thread Author':str,
                                'Domain':str, 'Expanded URLs':str,
                                'Avatar':str, 'Parent Blog Name':str, 'Root Blog Name':str},
                         usecols=['Date', 'Author', 'Full Text', 'Title', 'Thread Id',
                                  'Thread Author', 'Domain', 'Expanded URLs', 'Url']
                         )
        # df = df[['Date', 'Hashtags', 'Twitter Author ID', 'Author', 'Url', 'Thread Id', 'Thread Author', 'Domain']]
        df = df.rename(columns=brandwatch_column_dict)
        df_list.append(df)
        
    start_time = time.time() 
    result_df = pd.concat(df_list)
    end_time = time.time() 
    print(f"{(end_time - start_time)/60} mins for concat dataframes")
    
    start_time = time.time() 
    result_df.drop_duplicates(subset='source_msg_id', keep="first", inplace=True)
    end_time = time.time() 
    print(f"{(end_time - start_time)/60} mins for drop duplicates")
    
    start_time = time.time()
    if SAVE_OUTPUT_FILES:
        result_df['platform'].value_counts().rename('users_count').rename_axis('platform').to_csv(OUTPUT_PLATFORM_COUNTS_FILE)
    result_df = result_df[result_df['platform'].isin(filter_platform_domain_set)]
    end_time = time.time() 
    print(f"{(end_time - start_time)/60} mins for filtering platforms")
    
    result_df.reset_index(drop=True, inplace=True)
    print(result_df.shape)
    return result_df

In [ ]:
def read_brandwatch_data(data_directory, filter_platform_domain_set, is_using_s3):
    """
    Reads data from all the csv files in the given directory
    :param data_directory: Path to the directory that contains the csv files
    :type data_directory: str
    :return: pandas Dataframe that contains all the data from all csv files
    :rtype: pd.Dataframe
    """
    data_files = []
    if is_using_s3:
        data_files = s3.glob(os.path.join(data_directory, "*.csv*"))
    else:
        data_files = glob.glob(os.path.join(data_directory, "*.csv*"))
    prefix_path = ''
    if is_using_s3:
        prefix_path = 's3://'
    print(data_files)
    df_list = []
    for idx, file in enumerate(data_files):
        print(f"Reading {idx + 1} of {len(data_files)} files.\nFile name: {file}")
        df = pd.read_csv(prefix_path + data_files[idx], skiprows=6, parse_dates=['Date'],
                         dtype={'Twitter Author ID': str, 'Author':str,
                                'Full Text':str, 'Title':str,
                                'Thread Id':str, 'Thread Author':str,
                                'Domain':str, 'Expanded URLs':str,
                                'Avatar':str, 'Parent Blog Name':str, 'Root Blog Name':str},
                         usecols=['Date', 'Author', 'Full Text', 'Title', 'Thread Id',
                                  'Thread Author', 'Domain', 'Expanded URLs', 'Url']
                         )
        # df = df[['Date', 'Hashtags', 'Twitter Author ID', 'Author', 'Url', 'Thread Id', 'Thread Author', 'Domain']]
        df = df.rename(columns=brandwatch_column_dict)
        df_list.append(df)
        
    start_time = time.time() 
    result_df = pd.concat(df_list)
    end_time = time.time() 
    print(f"{(end_time - start_time)/60} mins for concat dataframes")
    
    start_time = time.time() 
    result_df.drop_duplicates(subset='source_msg_id', keep="first", inplace=True)
    end_time = time.time() 
    print(f"{(end_time - start_time)/60} mins for drop duplicates")
    
    start_time = time.time()
    if SAVE_OUTPUT_FILES:
        result_df['platform'].value_counts().rename('users_count').rename_axis('platform').to_csv(OUTPUT_PLATFORM_COUNTS_FILE)
    result_df = result_df[result_df['platform'].isin(filter_platform_domain_set)]
    end_time = time.time() 
    print(f"{(end_time - start_time)/60} mins for filtering platforms")
    
    result_df.reset_index(drop=True, inplace=True)
    print(result_df.shape)
    return result_df

In [ ]:
%%time
# read data and filter for only the KNOWN_PLATFORMS
all_osn_msgs_df = read_brandwatch_data(INPUT_BRANDWATCH_DIR, KNOWN_PLATFORMS, RUNNING_IN_S3)
all_osn_msgs_df

In [ ]:
#all_osn_msgs_df = all_osn_msgs_df[ourcols]
all_osn_msgs_df

In [ ]:
all_osn_msgs_df = all_osn_msgs_df[(all_osn_msgs_df["datetime"] >= datetime.datetime(2018,3,1)) & (all_osn_msgs_df["datetime"] <= datetime.datetime(2018,5,1))].reset_index(drop=True)
all_osn_msgs_df

In [ ]:
all_osn_msgs_df[all_osn_msgs_df["source_user_id"] == "sodrugestvo2"].set_index("datetime")

In [ ]:
min_time = all_osn_msgs_df["datetime"].min()
print(min_time)
max_time = all_osn_msgs_df["datetime"].max()
print(max_time)
#

In [ ]:
standard_index = pd.DatetimeIndex(pd.date_range(start=datetime.datetime(2018,3,1), end=datetime.datetime(2018,5,1), freq='H'))
standard_index

In [ ]:
dtmsg = all_osn_msgs_df.set_index("datetime")[["source_user_id","source_msg_id"]]
dtmsg

In [ ]:
all_osn_msgs_df[all_osn_msgs_df["source_user_id"] == "sodrugestvo2"].set_index("datetime")["source_user_id"].resample('15D', origin=datetime.datetime(2018,3,1)).count().reindex(standard_index, fill_value=0)

In [ ]:
dtmsg[dtmsg["source_user_id"] == "sodrugestvo2"].resample('15D', origin=datetime.datetime(2018,3,1)).count().reindex(standard_index, fill_value=0)

In [ ]:
requested_data = dtmsg.groupby("source_user_id").apply(lambda g: g.resample('H', origin=datetime.datetime(2018,3,1)).count().reindex(standard_index, fill_value=0))["source_msg_id"].rename_axis(['username','timeval']).reset_index()
requested_data

In [ ]:
pvt = requested_data.pivot(index="timeval", columns="username", values="source_msg_id")
pvt

In [ ]:
pvt.to_csv("user_post_by_hour.csv")

In [ ]:
requested_data

In [ ]:
5

In [ ]:
%%time
# add user_id column and parent_user_id column

def generate_users_dict(in_osn_msgs_df):
    global next_user_id
    users_data = {}
    next_user_id = 0
    def extract_user(row):
        global next_user_id
        if (row['platform'], row['source_user_id']) not in users_data:
            users_data[(row['platform'], row['source_user_id'])] = next_user_id
            next_user_id += 1
        if (row['platform'], row['parent_source_user_id']) not in users_data:
            users_data[(row['platform'], row['parent_source_user_id'])] = next_user_id
            next_user_id += 1
    in_osn_msgs_df.apply(lambda row: extract_user(row), axis=1)
    print(len(users_data))
    return users_data

users_data = generate_users_dict(all_osn_msgs_df)

users_df = pd.Series(users_data).rename_axis(['platform','source_user_id']).rename('user_id').reset_index()
print(users_df.shape)
print(users_df)
if SAVE_OUTPUT_FILES:
    users_df.to_csv(OUTPUT_USERS_FILE,index=False)

# add user_id column and parent_user_id column
all_osn_msgs_df[['user_id','parent_user_id']] = all_osn_msgs_df.apply(lambda row: pd.Series([
            users_data[(row['platform'],row['source_user_id'])],
            users_data[(row['platform'],row['parent_source_user_id'])]
        ]), axis = 1)

In [ ]:
%%time
# Add news_domains column

def find_patterns_for_domains(in_news_domains_csv_file, in_skip_strings):
    news_domains = pd.read_csv(in_news_domains_csv_file)['news outlets'].rename('news_outlets')
    news_domains_set = set(news_domains.to_list())
    pattern_to_news_domain_name = {re.compile(nd):nd for nd in news_domains_set}
    for nd in news_domains_set:
        valid_split_strs = set(nd.split('.'))
        for e in in_skip_strings:
            valid_split_strs.discard(e)
        for sp in valid_split_strs:
            if len(sp) > 2:
                pattern_to_news_domain_name[re.compile(sp)] = nd
    return pattern_to_news_domain_name

def search_domain_in_string(in_expanded_url, in_news_domains_names):
    # print(in_expanded_url)
    max_len_match = None
    max_len_found = 0
    for ndn in in_news_domains_names:
        match_obj = ndn.search(in_expanded_url)
        if match_obj:
            # print(match_obj, in_news_domains_names[ndn], match_obj.end() - match_obj.start())
            if max_len_found < match_obj.end() - match_obj.start():
                max_len_match = ndn
                max_len_found = match_obj.end() - match_obj.start()
    return in_news_domains_names[max_len_match] if max_len_match is not None else None


def calculate_news_domain_series(in_string_series, in_skip_strings):
    news_domains_names = find_patterns_for_domains(NEWS_DOMAINS_CSV_FILE, in_skip_strings)
    return in_string_series.apply(lambda x: search_domain_in_string(x, news_domains_names) if type(x) is str else None)

all_osn_msgs_df['news_domain'] = calculate_news_domain_series(all_osn_msgs_df['article_url'], skip_strings)
all_osn_msgs_df = all_osn_msgs_df.loc[all_osn_msgs_df['news_domain'].notnull(), :]

In [ ]:
link_regex = re.compile("((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)", re.DOTALL)

In [ ]:
links = re.findall(link_regex, df['content'].iloc[100])
links

In [ ]:
df['content'].iloc[100]

In [ ]:
links[0][0]

In [ ]:
4+5

In [ ]:
ndi.find_all_matches(df['content'].iloc[100])

In [ ]:
pattern_to_news_domain_name

In [ ]:
find_all_matches(df['content'].iloc[100], pattern_to_news_domain_prio)

In [ ]:
# add TUFMClass to osn messages table
all_osn_msgs_df['class'] = all_osn_msgs_df['news_domain'].apply(lambda x: news_domain_to_tufm_class[x] if x in news_domain_to_tufm_class else 'XX')
all_osn_msgs_df = all_osn_msgs_df[all_osn_msgs_df['class'] != 'XX']
print(all_osn_msgs_df['class'].unique())

# # filter by required news urls
# # filtering_urls = ['https://sputniknews.com/europe/201804171063638574-us-allies-skripal-case/',
# #  'https://www.independent.co.uk/news/uk/crime/salisbury-polisoning-latest-police-security-barriers-cordons-sergei-skripal-nerve-agent-a8308456.html',
# #  'http://www.bbc.co.uk/news/uk-43798068',
# #  'http://www.bbc.com/news/uk-43798068',
# #  'https://www.theguardian.com/uk-news/2018/apr/17/nerve-agent-used-to-attack-sergei-skripal-was-liquid-says-defra']
# #all_osn_msgs_df[all_osn_msgs_df['article_url'].isin(filtering_urls)]
# uoi = ['https://foreignpolicy.com/2018/03/27/britain-has-no-clue-why-its-punishing-russia/',
#        'https://www.nytimes.com/2018/03/29/world/europe/britain-yulia-skripal-hospital.html?smid=tw-nytimesworld&smtyp=cur',
#        'https://www.nbcnews.com/news/world/russian-ex-spy-says-he-was-kremlin-hit-list-along-n860641?cid=sm_npd_ms_tw_ma',
#        'https://www.nbcnews.com/news/world/russian-ex-spy-says-he-was-kremlin-hit-list-along-n860641?cid=sm_npd_nn_tw_ma',
#        'https://www.nbcnews.com/news/world/russian-ex-spy-says-he-was-kremlin-hit-list-along-n860641',
#        'https://www.nbcnews.com/news/world/yulia-skripal-daughter-russian-ex-spy-poisoned-u-k-no-n861141?cid=sm_npd_nn_tw_ma',
#        'https://wtop.com/j-j-green-national/2018/03/kremlin-deflects-blame-skripal-poisoning-accuses-businessman/',
#        'https://voxpoliticalonline.com/2018/03/29/nerve-agent-smeared-in-large-quantities-on-skripal-front-door-then-how-is-yulia-getting-better/',
#        'https://www.theguardian.com/uk-news/2018/mar/29/russia-accuses-uk-of-violating-international-law-skripal-case',
#        'https://www.nbcnews.com/news/world/yulia-skripal-daughter-russian-ex-spy-poisoned-u-k-no-n861141?cid=sm_npd_ms_tw_ma',
#        'http://www.bbc.co.uk/news/uk-43588450, https://www.theguardian.com/uk-news/2018/mar/29/russia-accuses-uk-of-violating-international-law-skripal-case',
#        'http://www.bbc.com/news/uk-43588450',
#        'https://www.theguardian.com/uk-news/2018/mar/28/skripals-poisoned-from-front-door-of-salisbury-home-police-say?CMP=twt_gu',
#        'https://www.independent.co.uk/news/uk/crime/sergei-skripal-salisbury-poison-nerve-agent-russia-daughter-attack-novichok-front-door-home-a8278631.html',
#        'https://www.theguardian.com/uk-news/2018/mar/29/russia-accuses-uk-of-violating-international-law-skripal-case?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1522348151',
#        'https://www.theguardian.com/uk-news/2018/mar/29/yulia-skripal-no-longer-in-critical-condition-says-salisbury-nhs?CMP=share_btn_tw',
#        'https://www.theguardian.com/uk-news/2018/mar/29/yulia-skripal-no-longer-in-critical-condition-says-salisbury-nhs?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1522348157',
#        'https://twitter.com/bbcnews/status/979363446054621184',
#        'https://www.theguardian.com/uk-news/2018/mar/29/yulia-skripal-no-longer-in-critical-condition-says-salisbury-nhs',
#        'http://www.bbc.co.uk/news/uk-43588450',
#        'http://www.euronews.com/2018/03/29/russian-denials-over-skripal-ludicrous-claims-johnson?utm_campaign=Echobox&utm_medium=Social&utm_source=Twitter#link_time=1522340777',
#        'https://www.independent.co.uk/news/uk/crime/yulia-skripal-latest-nerve-agent-critical-condition-salisbvury-poisoning-russia-spy-a8279761.html',
#        'https://twitter.com/bbcdaniels/status/979359954552479744',
#        'https://www.rt.com/uk/422560-national-security-review-propaganda/',
#        'https://www.nytimes.com/2018/03/28/world/europe/poison-skripal-russia.html?smid=tw-nytimes&smtyp=cur',
#        'https://www.nbcnews.com/news/world/police-say-russian-ex-spy-sergei-skripal-daughter-likely-poisoned-n860966?cid=sm_npd_nn_tw_ma',
#        'https://www.thedailybeast.com/sergei-skripal-was-likely-poisoned-by-nerve-agent-on-his-front-door?via=twitter_page',
#        'http://www.bbc.co.uk/news/uk-43577987',
#        'https://www.bloomberg.com/view/articles/2018-03-28/the-west-s-response-to-russia-after-skripal-attack-is-too-mild',
#        'https://www.independent.co.uk/news/uk/crime/sergei-skripal-salisbury-poison-nerve-agent-russia-daughter-attack-novichok-front-door-home-a8278631.html?utm_campaign=Echobox&utm_medium=Social&utm_source=Twitter',
#        'https://www.rt.com/news/422514-zeman-skripal-uk-facts/',
#        'http://www.dailymail.co.uk/news/article-5552693/Sergei-Skripal-daughter-allowed-die.html',
#        'https://www.rt.com/news/422529-uk-embassy-presentation-skripal/',
#        'https://talkingpointsmemo.com/world-news/daughter-ex-russian-spy-poisoned-nerve-agent-attack-out-of-critical-condition',
#        'http://www.dailymail.co.uk/news/article-5558445/Investigators-remove-poisoned-spy-Sergei-Skripals-door.html#ixzz5BEST2ENe',
#        'https://www.facebook.com/bbcnewsnight/videos/10155465289046200/',
#        'https://www.thetimes.co.uk/edition/news/hopes-rise-as-yulia-skripal-is-taken-off-critical-list-nm52vvdrj?CMP=Sprkr-_-Editorial-_-thetimes-_-Unspecified-_-TWITTER',
#        'http://www.dailymail.co.uk/news/article-5573751/Skripal-family-call-Britain-make-clear-happened-ex-spys-beloved-pets.html',
#        'https://www.theguardian.com/uk-news/2018/apr/03/porton-down-experts-unable-to-verify-precise-source-of-novichok',
#        'http://skwawkbox.org/2018/04/03/of-course-bbcs-coverage-of-johnsons-novichoklie-is-fine-no-need-to-share-this-video/',
#        'https://www.rt.com/uk/423075-porton-down-skripal-proof/',
#        'http://www.dailymail.co.uk/news/article-5570327/Yulia-Skripal-poisoned-days-gaining-access-secret-150-000-bank-account.html',
#        'https://www.cnn.com/2018/04/03/europe/skripal-poisoning-uk-nerve-agent-intl/index.html?sr=twCNN040318skripal-poisoning-uk-nerve-agent-intl0352PMVODtop',
#        'https://apple.news/AvVsMpO-FSqaTb7sz01Xbcw',
#        'https://www.theguardian.com/world/2018/mar/22/andrei-zheleznyakov-soviet-scientist-poisoned-novichok',
#        'https://www.theguardian.com/uk-news/2018/apr/03/porton-down-experts-unable-to-verify-precise-source-of-novichok?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1522776092',
#        'https://www.nytimes.com/2018/04/01/world/europe/russia-sergei-skripal-uk-spy-poisoning.html?rref=collection%2Fsectioncollection%2Fworld&action=click&contentCollection=world&region=rank&module=package&version=highlights&contentPlacement=2&pgtype=sectionfront',
#        'http://www.newsweek.com/russian-defectors-america-putin-russia-threat-skripal-868013, https://twitter.com/danarohrabacher/status/980140617635278848',
#        'https://www.theguardian.com/uk-news/2018/apr/02/salisbury-attack-sites-decontamination-russia-sergei-skripal?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1522690078',
#        'https://twitter.com/npr/status/980815284515868673',
#        'https://www.thetimes.co.uk/edition/news/2-800-russian-bots-sowed-confusion-after-poison-attacks-zf6lvb3nc',
#        'https://www.washingtonpost.com/world/trump-proposed-putin-visit-white-house-in-march-20-phone-call-kremlin-says/2018/04/02/b0727634-367b-11e8-acd5-35eac230e514_story.html',
#        'https://www.washingtonpost.com/news/made-by-history/wp/2018/04/02/the-real-reason-russia-blames-britain-for-the-skripal-poisonings/?utm_term=.8208820aed18&tid=sm_tw',
#        'https://www.nytimes.com/2018/04/01/world/europe/russia-sergei-skripal-uk-spy-poisoning.html?smid=tw-share',
#        'https://edition.cnn.com/2018/04/02/europe/skripal-poisoning-uk-kremlin-approval-intl/index.html?sr=twCNN040218skripal-poisoning-uk-kremlin-approval-intl0751AMVODtop',
#        'https://www.independent.co.uk/news/world/europe/sergei-skripal-latest-salisbury-poisoning-attack-russia-nerve-agent-sergei-lavrov-a8284766.html?utm_campaign=Echobox&utm_medium=Social&utm_source=Twitter',
#        'https://www.nytimes.com/2018/04/01/world/europe/russia-sergei-skripal-uk-spy-poisoning.html',
#        'https://www.cbsnews.com/news/russian-state-tv-airs-alleged-phone-call-from-yulia-skripal-daughter-of-poisoned-ex-spy/',
#        'https://www.independent.co.uk/news/world/europe/salisbury-spy-poisoning-latest-russia-uk-sergei-skripal-un-ambassador-karen-pierce-a8291086.html',
#        'http://www.bbc.co.uk/news/av/world-europe-43663007/russian-tv-airs-skripal-phone-call',
#        'https://www.theguardian.com/uk-news/video/2018/apr/05/russian-tv-broadcasts-call-from-yulia-skripal-video?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1522946048',
#        'https://www.reuters.com/article/us-britain-russia-skripal-yulia-health/daughter-of-ex-spy-poisoned-in-uk-says-she-and-her-father-are-recovering-russian-state-tv-idUSKCN1HC1EU?utm_campaign=trueAnthem:+Trending+Content&utm_content=5ac68c0704d301294e9725b5&utm_medium=trueAnthem&utm_source=twitter',
#        'https://www.theguardian.com/uk-news/2018/apr/05/recovery-of-yulia-skripal-injects-personality-into-uk-and-russias-script?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1522958647',
#        'https://www.independent.co.uk/news/uk/home-news/salisbury-spy-poisoning-latest-russia-uk-sergei-skripal-playing-with-fire-a8291051.html?utm_campaign=Echobox&utm_medium=Social&utm_source=Twitter',
#        'http://www.cbc.ca/news/world/daughter-poisoning-spy-yulia-sergei-skripal-uk-russia-1.4606047',
#        'http://www.bbc.co.uk/news/av/world-europe-43663007/russian-tv-airs-skripal-phone-call?ocid=socialflow_twitter',
#        'https://www.theguardian.com/commentisfree/2018/apr/05/boris-johnson-intelligence-skripal-poisoning-security?CMP=share_btn_tw',
#        'https://www.theguardian.com/uk-news/2018/apr/05/yulia-skripal-says-everyone-is-getting-better-russian-tv-reports?CMP=share_btn_tw',
#        'https://www.nbcnews.com/news/weird-news/yulia-skripal-daughter-ex-russian-spy-gives-first-statement-after-n862901',
#        'http://www.bbc.co.uk/news/world-europe-43652574',
#        'https://www.theguardian.com/uk-news/2018/apr/05/yulia-skripal-says-everyone-is-getting-better-russian-tv-reports?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1522935147',
#        'https://www.bloomberg.com/news/articles/2018-04-05/woman-in-russian-spy-mystery-moves-to-center-of-diplomatic-spat?utm_source=twitter&cmpid=socialflow-twitter-business&utm_content=business&utm_campaign=socialflow-organic&utm_medium=social',
#        'https://www.washingtonpost.com/news/worldviews/wp/2018/03/12/what-is-novichok-the-russian-nerve-agent-and-the-scientist-who-revealed-it/?utm_term=.34d91eea4ca6',
#        'https://www.thetimes.co.uk/article/3c86d182-3853-11e8-9a8f-0b0aae019371',
#        'https://www.reuters.com/article/us-britain-russia-skripal-yulia-health/daughter-of-ex-spy-poisoned-in-uk-says-she-and-her-father-are-recovering-russian-state-tv-idUSKCN1HC1EU',
#        'https://www.theguardian.com/uk-news/2018/apr/05/yulia-skripal-says-everyone-is-getting-better-russian-tv-reports',
#        'https://www.thetimes.co.uk/article/useful-idiots-are-letting-putin-off-the-hook-fz63rn9g2',
#        'http://www.bbc.co.uk/news/world-europe-43652574?ns_mchannel=social&ns_campaign=bbc_breaking&ns_source=twitter&ns_linkname=news_central',
#        'https://www.thetimes.co.uk/article/may-battles-to-preserve-alliance-against-russia-qcwkzlt63?shareToken=f0eb79b8f82f18a7fbe2d9c4f4499007',
#        'https://www.independent.co.uk/news/uk/politics/british-army-could-stage-mutiny-under-corbyn-says-senior-serving-general-10509742.html',
#        'https://www.facebook.com/bbcnews/videos/10155465941646200/',
#        'https://www.cbsnews.com/news/sergei-skripal-former-russian-spy-no-longer-in-critical-condition/?ftag=CNM-00-10aab7e&linkId=50155694',
#        'https://www.theguardian.com/world/video/2018/apr/06/sergei-skripal-responding-well-to-treatment-says-salisbury-hospital?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1523050172',
#        'https://www.independent.co.uk/news/uk/politics/viktoria-skripal-uk-visa-denied-home-office-sergei-salisbury-visit-hospital-yulia-a8292481.html',
#        'https://www.rt.com/uk/423397-skripal-hospital-critical-salisbury/',
#        'http://www.dailymail.co.uk/news/article-5586087/Sergei-Skripal-no-longer-critical-condition-according-doctors.html',
#        'https://www.independent.co.uk/news/uk/politics/victoria-skripal-uk-visa-denied-home-office-sergei-salisbury-visit-hospital-yulia-a8292481.html',
#        'https://www.theguardian.com/uk-news/2018/apr/06/salisbury-poisoning-what-next-now-sergei-skripal-is-no-longer-critically-ill?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1523045068',
#        'https://www.theguardian.com/commentisfree/2018/apr/06/boris-johnson-post-truth-post-shame-novichok',
#        'https://www.independent.co.uk/voices/sergei-skripal-russia-nerve-agent-novichok-recovery-a8292726.html?utm_campaign=Echobox&utm_medium=Social&utm_source=Twitter',
#        'https://www.theguardian.com/uk-news/2018/apr/06/salisbury-poisoning-what-next-now-sergei-skripal-is-no-longer-critically-ill?CMP=twt_gu',
#        'https://www.nbcnews.com/news/world/sergei-skripal-improving-rapidly-nerve-agent-poison-attack-n863291?cid=sm_npd_ms_tw_ma',
#        'https://www.theguardian.com/uk-news/2018/apr/06/sergei-skripal-no-longer-in-critical-condition-and-improving-rapidly?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1523028092',
#        'http://time.com/5229321/sergei-skripal-yulia-russian-spy-poisoning/?utm_campaign=time&utm_source=twitter.com&utm_medium=social&xid=time_socialflow_twitter',
#        'https://www.theguardian.com/us-news/2018/apr/05/ex-trump-aide-paul-manafort-approved-black-ops-to-help-ukraine-president',
#        'https://www.nytimes.com/2018/04/05/world/europe/yulia-skripal-russia-britain.html',
#        'https://www.nbcnews.com/news/world/yulia-skripal-daughter-ex-russian-spy-gives-first-statement-after-n862901?cid=sm_npd_nn_tw_ma',
#        'https://www.cbsnews.com/news/poisoned-russian-ex-spy-sergei-skripal-daughter-yulia-skripal-speaks/?ftag=COS-05-10aaa0g&utm_campaign=trueAnthem:+Trending+Content&utm_content=5ac70d9504d301097272070c&utm_medium=trueAnthem&utm_source=twitter',
#        'https://www.cbsnews.com/news/sergei-skripal-poisoning-britain-believes-it-has-found-russian-lab-that-made-nerve-agent-salisbury/?ftag=COS-05-10aaa0g&utm_campaign=trueAnthem:+Trending+Content&utm_content=5ac70a1204d301796b5c063f&utm_medium=trueAnthem&utm_source=twitter',
#        'https://www.cbsnews.com/news/russian-state-tv-airs-alleged-phone-call-from-yulia-skripal-daughter-of-poisoned-ex-spy/?ftag=CNM-00-10aab7e&linkId=50113818',
#        'http://www.dailymail.co.uk/news/article-5590067/Nerve-agent-used-poison-Russian-spy-designed-4-hours-work-allow-culprits-flee.html#ixzz5C3pWySxj',
#        'https://www.theguardian.com/politics/2018/apr/08/theresa-may-scandinavia-row-russia-salisbury-attack-skripal-nerve-agent?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1523223342',
#        'https://www.theguardian.com/uk-news/2018/apr/04/uk-urged-to-counter-disinformation-from-russia-over-novichok?CMP=Share_iOSApp_Other',
#        'http://skwawkbox.org/2018/04/08/video-bbc-covers-for-johnson-again-as-marr-loses-johnsons-novichok-source-question/',
#        'https://www.washingtonpost.com/news/worldviews/wp/2018/04/06/russia-claims-a-poisoned-spys-dead-cat-and-guinea-pigs-hold-clues-to-british-conspiracy/?utm_term=.92b656a7d6cd',
#        'https://iz.ru/729715/2018-04-08/gp-obnaroduet-svidetelstva-nezakonnogo-polucheniia-berezovskim-ubezhishcha-v-velikobritanii',
#        'http://bulgarianpresidency.eu/poll-88-of-bulgarians-side-with-russia-in-the-skripal-case/',
#        'http://www.dailymail.co.uk/news/article-5581507/Yulia-Skripal-calls-Moscow-says-leave-hospital-days.html',
#        'https://www.thedailybeast.com/report-sergei-skripal-daughter-to-get-new-identities',
#        'https://www.independent.co.uk/news/uk/home-news/sergei-skripal-pets-cat-guinea-pig-deaths-salisbury-nerve-agent-peta-a8292251.html',
#        'http://www.dailymail.co.uk/news/article-5590007/Niece-Sergei-Skripal-goads-Theresa-video-message.html',
#        'https://sputniknews.com/analysis/201804051063242381-skripal-false-flag-uk/?utm_source=short_direct&utm_medium=short_url&utm_content=hjuN&utm_campaign=URL_shortening',
#        'https://www.nytimes.com/2018/04/05/world/europe/russia-landfills-gases.html?smid=tw-nytimes&smtyp=cur',
#        'https://www.newstatesman.com/world/2018/04/johnson-corbyn-and-too-many-others-are-playing-russia-s-hands-skripal-poisoning',
#        'https://twitter.com/moscowtimes/status/977093642568552448',
#        'http://eastwestblog.net/2016/05/14/ukraines-parliament-asks-bbc-to-stop-using-civil-war-for-russias-aggression/',
#        'https://www.reuters.com/article/us-britain-russia-skripal-usa/britain-aims-to-resettle-poisoned-russian-ex-spy-in-the-u-s-sunday-times-idUSKBN1HE0TY?utm_campaign=trueAnthem:+Trending+Content&utm_content=5ac9522704d3014f4885143f&utm_medium=trueAnthem&utm_source=twitter',
#        'http://www.bbc.co.uk/news/uk-43680216',
#        'https://www.zerohedge.com/news/2018-04-07/skripal-gate-collapses-so-will-may-government',
#        'https://www.thetimes.co.uk/article/novichok-scientist-vil-mirzayanov-reveals-decades-of-lies-3s5rglflp?CMP=Sprkr-_-Editorial-_-thetimes-_-Unspecified-_-TWITTER',
#        'https://www.reuters.com/article/us-britain-russia-skripal-asylum/yulia-skripal-to-seek-political-asylum-ria-citing-cousin-idUSKBN1HG2I1?utm_campaign=trueAnthem:+Trending+Content&utm_content=5acbaeda04d3013ab4a944f5&utm_medium=trueAnthem&utm_source=twitter',
#        'http://www.bbc.co.uk/news/uk-43710126?ns_mchannel=social&ns_campaign=bbc_breaking&ns_source=twitter&ns_linkname=news_central',
#        'https://www.reuters.com/article/us-britain-russia-skripal/poisoned-yulia-skripal-has-been-discharged-from-british-hospital-bbc-idUSKBN1HH0PM?feedType=RSS&feedName=topNews&utm_source=twitter&utm_medium=Social',
#        'https://www.independent.co.uk/news/uk/home-news/yulia-skripal-latest-health-hospital-sergei-russia-spy-novichok-salisbury-a8297136.html?utm_campaign=Echobox&utm_medium=Social&utm_source=Twitter',
#        'https://www.reuters.com/article/us-britain-russia-skripal/yulia-skripal-to-be-discharged-from-british-hospital-on-tuesday-sky-news-says-idUSKBN1HH0PM?feedType=RSS&feedName=topNews&utm_medium=Social&utm_source=twitter',
#        'https://www.thetimes.co.uk/article/russian-message-intercepted-after-skripal-attack-6t2mkbx0v?CMP=Sprkr-_-Editorial-_-thetimes-_-Unspecified-_-TWITTER',
#        'https://www.theguardian.com/politics/2018/apr/15/boris-johnson-defends-remarks-on-russian-role-in-novichok-case?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1523801329',
#        'http://www.dailymail.co.uk/news/article-5616859/Novichok-Salisbury-hitman-failed-kill-Skripals-raining.html',
#        'http://www.dailymail.co.uk/news/article-5616611/Sinister-social-media-post-Yulia-Skripals-Russian-Facebook-site-raises-new-hacking-fears.html',
#        'https://www.rt.com/news/424149-skripal-poisoning-bz-lavrov/',
#        'https://www.independent.co.uk/news/uk/home-news/skripals-skripal-russia-nerve-agent-bz-novichok-poison-salisbury-attack-latest-update-a8304841.html',
#        'https://www.zerohedge.com/news/2018-04-14/independent-swiss-lab-says-bz-toxin-used-skripal-poisoning-usuk-produced-not',
#        'https://thebell.io/en/the-scientist-who-developed-novichok-doses-ranged-from-20-grams-to-several-kilos/',
#        'https://uk.reuters.com/article/uk-britain-russia-skripal-lavrov/russias-lavrov-says-skripals-may-have-been-poisoned-by-substance-russia-never-made-idUKKBN1HL17G',
#        'https://www.economist.com/news/britain/21740776-mischievous-englishwomen-are-blamed-everything-skripal-poisoning-17th-century?fsrc=scn/tw/te/bl/ed/kremlinbackedfakenewstargetsbritainanglichankastrikesagain',
#        'https://www.nbcnews.com/tech/tech-news/russian-propaganda-evades-youtube-s-flagging-system-buzzfeed-style-knockoffs-n867431',
#        'https://www.nytimes.com/2018/03/12/world/europe/uk-russia-spy-poisoning.html',
#        'https://www.cbsnews.com/news/russia-poisoned-ex-spy-sergei-skripal-daughter-yulia-airport-video-moscow/?ftag=CNM-00-10aab7e&linkId=50685865',
#        'https://www.thetimes.co.uk/edition/news/salisbury-clean-up-after-skripal-poisoning-will-take-months-t7h3kcxh8?CMP=Sprkr-_-Editorial-_-thetimes-_-Unspecified-_-TWITTER',
#        'https://www.reuters.com/article/us-britain-russia/russias-rt-faces-seven-investigations-in-britain-over-broadcasts-since-skripal-case-idUSKBN1HP1CM?feedType=RSS&feedName=topNews&utm_medium=Social&utm_source=twitter',
#        'https://www.thedailybeast.com/rts-skripal-coverage-investigated-by-uk-tv-watchdog?source=twitter&via=mobile',
#        'http://www.bbc.com/news/uk-43805167?ns_source=twitter&ocid=socialflow_twitter&ns_campaign=bbcnews&ns_mchannel=social',
#        'https://www.theguardian.com/uk-news/2018/apr/17/nerve-agent-used-to-attack-sergei-skripal-was-liquid-says-defra',
#        'http://www.bbc.com/news/uk-43798068',
#        'https://www.rt.com/shows/renegade-inc/424239-weapons-mass-distraction-issue/',
#        'https://www.economist.com/news/europe/21740456-smolensk-salisbury-conspiracy-theories-and-about-russia?fsrc=scn/tw/te/bl/ed/conspiracytheoriesfromandaboutrussiaflightsofimagination',
#        'https://www.theguardian.com/uk-news/2018/apr/17/nerve-agent-used-to-attack-sergei-skripal-was-liquid-says-defra?utm_term=Autofeed&CMP=twt_b-gdnnews#link_time=1523975218',
#        'http://www.bbc.com/news/uk-43798068?ns_campaign=bbcnews&ns_source=twitter&ns_mchannel=social&ocid=socialflow_twitter',
#        'https://monitoring.bbc.co.uk/product/c1dp2xz8',
#        'http://www.bbc.co.uk/news/uk-43798068']
# filtered_msgs = all_osn_msgs_df[all_osn_msgs_df['article_url'].isin(uoi)]


# filter by class
#all_osn_msgs_df[all_osn_msgs_df['class'].isin(['TM','UF'])]

filtered_msgs = all_osn_msgs_df
filtered_msgs

In [ ]:
#%%time
# generate retweet network
retweet_network_edges = filtered_msgs.groupby(['parent_user_id','user_id'], as_index=False).size().sort_values('size', ascending=False).rename(columns={'size':'num_retweets'})
print(retweet_network_edges)
if SAVE_OUTPUT_FILES:
    retweet_network_edges.to_csv(OUTPUT_RETWEET_NETWORK_FILE, index=False)

In [ ]:
#%%time
# generate community partition on retweet network
G = nx.from_pandas_edgelist(retweet_network_edges,'parent_user_id','user_id',['num_retweets'])
print(G)
lc = nx.algorithms.community.louvain_communities(G, resolution=1, seed=235246345345)
print(len(lc))
user_comm_sizes = pd.Series([len(c) for c in lc]).value_counts().rename_axis('community_size').rename('num_of_communities').reset_index().sort_values('community_size', ascending=False).reset_index(drop=True).rename_axis('size_rank')
if SAVE_OUTPUT_FILES:
    user_comm_sizes.to_csv(OUTPUT_ALL_USER_COMM_SIZES_FILE)
print(user_comm_sizes[user_comm_sizes['community_size'] > 500])
filtered_comms = [c for c in lc if len(c) > MIN_COMM_SIZE]
print(f"Number of communities with min of {MIN_COMM_SIZE} users : {len(filtered_comms)}")

In [ ]:
#%%time
# generate actor tables 

# generate platform actors table
platform_to_user_count = filtered_msgs['platform'].value_counts().to_dict()
platform_actors_df = pd.Series(platform_to_user_count).rename('users_count').rename_axis('platform').reset_index()
platform_actors_df = platform_actors_df[(platform_actors_df['users_count'] > MIN_PLAT_SIZE) & (platform_actors_df['platform'].isin(KNOWN_PLATFORMS))].reset_index(drop=True).rename_axis('actor_id')
next_actor_id = platform_actors_df.index.max() + 1
print(next_actor_id)
print("plat_actors:\n", platform_actors_df, "\n")
if SAVE_OUTPUT_FILES:
    platform_actors_df.to_csv(OUTPUT_PLATFORM_ACTORS_FILE)

# generate individual actors table
ind_actors_df = filtered_msgs['parent_user_id'].value_counts().iloc[:100].rename('received_share_count').rename_axis('user_id').reset_index().rename_axis('actor_id')
ind_actors_df.index = ind_actors_df.index + next_actor_id
ind_actors_df.reset_index(inplace=True)
next_actor_id = ind_actors_df['actor_id'].max() + 1
print(next_actor_id)
print("indv_actors:\n", ind_actors_df, "\n")
if SAVE_OUTPUT_FILES:
    ind_actors_df.to_csv(OUTPUT_INDV_ACTORS_FILE, index=False)



# generate community actors table
comm_info_df = pd.DataFrame([[len(c), c] for c in filtered_comms], columns=['users_count', 'users_set']).sort_values('users_count', ascending=False).reset_index(drop=True).rename_axis('comm_id')
comm_info_df['actor_id'] = comm_info_df.index + next_actor_id
comm_info_df = comm_info_df.reset_index().set_index('actor_id')
if SAVE_OUTPUT_FILES:
    comm_info_df.to_csv(OUTPUT_COMM_INFO_FILE)
print("Community info :\n", comm_info_df)
comm_actors_data = []
for actid,row in comm_info_df.iterrows():
    for uid in row['users_set']:
        comm_actors_data.append([uid, row['comm_id'], actid])
        
comm_actors_df = pd.DataFrame(comm_actors_data, columns=['user_id','comm_id','actor_id'])
print("comm_actors:\n", comm_actors_df, "\n")
if SAVE_OUTPUT_FILES:
    comm_actors_df.to_csv(OUTPUT_COMM_ACTORS_FILE, index=False)


# generate actors table
all_actors = [[plt_aid, platform_actors_df.loc[plt_aid]['platform'], 'plat', platform_actors_df.loc[plt_aid]['users_count']] for plt_aid in platform_actors_df.index.to_list()]
all_actors = all_actors + [[indv,
                            users_df[ users_df['user_id'] == ind_actors_df[ind_actors_df['actor_id']==indv].iloc[0]['user_id'] ].iloc[0]['source_user_id'],
                            'indv', 1] for indv in ind_actors_df['actor_id']]
print('here')
all_actors = all_actors + list({(comm_aid, comm_info_df.loc[comm_aid]['comm_id'], 'comm', comm_info_df.loc[comm_aid]['users_count']) for comm_aid in comm_info_df.index})
print('here')
all_actors_df = pd.DataFrame(all_actors, columns=['actor_id','actor_label','actor_type','num_users']).sort_values('actor_id')
if SAVE_OUTPUT_FILES:
    all_actors_df.to_csv(OUTPUT_ACTORS_FILE, index=False)
all_actors_df

In [ ]:
if SAVE_OUTPUT_FILES:
    all_osn_msgs_df.to_csv(OUTPUT_ALL_OSN_MSGS_FILE, index=False)
all_osn_msgs_df

In [ ]:
# upload datafiles to s3 bucket
if UPLOAD_FILES:
    s3_bucket_object = boto3.Session().resource('s3').Bucket(UPLOAD_S3_BUCKET_NAME)
    for file_name in [os.path.basename(f) for f in glob.glob("./OUTPUT/*.csv")]:
        print(f"Uploading file : {OUTPUT_DIR}/{file_name} to {UPLOAD_S3_BUCKET_NAME}/{UPLOAD_S3_BUCKET_SUBFOLDER}/{file_name}")
        s3_bucket_object.upload_file(f"{OUTPUT_DIR}/{file_name}", f"{UPLOAD_S3_BUCKET_SUBFOLDER}/{file_name}")

In [ ]:
import importlib
importlib.reload(transfer_entropy_functions)

def compute_actor_te_network(actors_of_interest, t3_all_osn_msgs, t6_actors, t7_indv_actors, t8_comm_actors, t9_plat_actors, freq, start_date, end_date):
    # generate te network for the selected actors
    print(t6_actors)
    #start_date = t3_all_osn_msgs['datetime'].dt.date.min()
    #end_date = t3_all_osn_msgs['datetime'].dt.date.max() + datetime.timedelta(days=1)
    results_df = transfer_entropy_functions.generate_te_edge_list(actors_of_interest, t3_all_osn_msgs, t6_actors, t7_indv_actors, t8_comm_actors, t9_plat_actors, freq, start_date, end_date)
    return results_df

In [ ]:
#platform_actors_df

In [ ]:
#comm_actors_df

In [ ]:
#ind_actors_df

In [ ]:
#all_osn_msgs_df.iloc[541].to_dict()

In [ ]:
all_actors_df = all_actors_df.set_index('actor_id')
ind_actors_df = ind_actors_df.set_index('actor_id')
comm_actors_df = comm_actors_df.set_index('actor_id')

In [ ]:
# Get the moving/growing window start points for the given duration by the given shift size
print(f"Start Date : {START_DATE}")
print(f"End Date : {END_DATE}")
print(f"Window Shift By : {WINDOW_SHIFT_BY_DAYS}")
print(f"Init Window Size : {INIT_WINDOW_SIZE}")
dynamic_windows = pd.DatetimeIndex(pd.date_range(start=START_DATE, end=END_DATE, freq=f"{WINDOW_SHIFT_BY_DAYS}D", closed='left'))
print(dynamic_windows)

# Moving/Growing windows start and end dates
IS_GROWING_WINDOW = True
dwindows_df = pd.DataFrame( { 
    'start_date': pd.Series(dynamic_windows[0] if IS_GROWING_WINDOW else dw for dw in dynamic_windows),
    'end_date': pd.Series([dw + INIT_WINDOW_SIZE for dw in dynamic_windows])
              })
# remove days that are out of end date boundary
dwindows_df = dwindows_df[dwindows_df['end_date'] < END_DATE + datetime.timedelta(days=WINDOW_SHIFT_BY_DAYS) ]
dwindows_df

In [ ]:
# Iterate over all moving windows and compute the series of results

importlib.reload(transfer_entropy_functions)

results = []
for idx, row in dwindows_df.iterrows():
    print(idx)
    print(f"{row['start_date']} to {row['end_date']}")
    r = compute_actor_te_network(all_actors_df.index.to_list(), filtered_msgs, all_actors_df, ind_actors_df, comm_actors_df, platform_actors_df,
                                 SAMPLE_FREQUENCY, row['start_date'], row['end_date'])
    results.append(r)

In [ ]:
len(results)

In [ ]:
for i, r in enumerate(results):
    s = dwindows_df['start_date'].iloc[i].date()
    s = str(s).replace("-","_")
    print(f"r : {i} : {s}")
    print(f"\tnum of TE <= 0 edges : {r[r['total_te'] <= 0].shape[0]}")
    print(f"\tnum of TE > 0 edges : {r[r['total_te'] > 0].shape[0]}")
    r = r[r['total_te'] > 0]
    print(r.shape)
    print(r.head())

In [ ]:
for i,r in enumerate(results):
    s1 = dwindows_df['start_date'].iloc[i].date()
    s1 = str(s1).replace("-","_")
    s2 = dwindows_df['end_date'].iloc[i].date()
    s2 = str(s2).replace("-","_")
    print(f"r : {i} : {s}")
    if SAVE_OUTPUT_FILES:
        r.to_csv(OUTPUT_DIR + f"/dynamic/growing/actor_te_edges_{s1}_{s2}.csv", index=False)
        if UPLOAD_FILES:
            s3_bucket_object.upload_file(f"{OUTPUT_DIR}/dynamic/growing/actor_te_edges_{s1}_{s2}.csv", f"{UPLOAD_S3_BUCKET_SUBFOLDER}/dynamic/growing/actor_te_edges_{s1}_{s2}.csv")

In [ ]:
all_actors_df.actor_type.unique()

In [ ]:
classes=['UF', 'UM', 'TF', 'TM']
cols = []
for s in classes:
    for t in classes:
        cols.append(f"{s}_{t}")
ecols = []
for col in cols:
    c = f"e_{col}"
    ecols.append(c)
    r[c] = r[col].apply(lambda x: 1 if x > 0 else 0)
r

In [ ]:
if SAVE_OUTPUT_FILES:
    r[['Source', 'Target'] + ecols].to_csv(OUTPUT_DIR + '/gephi_actor_te_edges.csv', index=False)
    if UPLOAD_FILES:
        s3_bucket_object.upload_file(f"{OUTPUT_DIR}/actor_te_edges.csv", f"{UPLOAD_S3_BUCKET_SUBFOLDER}/gephi_actor_te_edges.csv")

In [ ]:
x = r[['Source', 'Target'] + ecols]
x

In [ ]:
import matplotlib.pyplot as plt
xs = []
hs = []
cmap = {'UF':'tomato','UM':'lawngreen','TF':'cornflowerblue','TM':'fuchsia'}
cs = []
for ec in ecols:
    print(f"{ec[2:]} -> {x[ec].sum()}")
    xs.append(ec[2:])
    hs.append(x[ec].sum())
    cs.append(cmap[ec[2:4]])
    
f, ax = plt.subplots(figsize=(10,7))
plt.bar(xs, hs, color=cs)
plt.xticks(rotation=60)
plt.title('Total Number of Edges')
plt.ylabel('Count')
plt.xlabel('Influence Class')
plt.show()

In [ ]:
d = filtered_msgs['class'].value_counts().to_dict()
print(d)

f, ax = plt.subplots(figsize=(5,7))
plt.bar(classes, [d[c] for c in classes], color=[cmap[c] for c in classes])
#plt.xticks(rotation=60)
plt.title('Total Number of Tweets')
plt.ylabel('Count')
plt.xlabel('News Domain Class')
plt.show()

In [ ]:
all_osn_msgs_df["article_url"].nunique()

In [ ]:
all_osn_msgs_df['news_domain'].value_counts()

In [ ]:
all_osn_msgs_df.

In [ ]:
gdf = all_osn_msgs_df[['datetime','news_domain']].reset_index().set_index('datetime')#.set_index('datetime').resample('D')#.Resampler.count()#apply(lambda x: pd.DataFrame(x))
print(gdf)

grouper = gdf.groupby([pd.Grouper(freq='1D'), 'news_domain'])
result = grouper['index'].count().unstack('news_domain').fillna(0)

result

In [ ]:
result[['theguardian.com','sputniknews.com','rt.com']].plot(figsize=(14,14), ylabel='Posts Count', xlabel='Time')

In [ ]:
result.columns.to_list()